In [1]:
import numpy as np
import tensorly as tl
from tensorly.decomposition import parafac
from numpy.linalg import matrix_rank, pinv, inv

np.set_printoptions(suppress=True)

## Choose parameters
(Maybe we can use some python package for handling graphs?)

In [2]:
nlat = 6
joint_latents = [0,1,2]
env1_specific = [3]
env2_specific = [4]
env3_specific = [5]
d1 = 12
d2 = 15
d3 = 12

# Third-order moments of the latent variables:
third_moms = [1,-2,3,4,4.5,4.2]
np.arange(1,(nlat+1))
third_moms

[1, -2, 3, 4, 4.5, 4.2]

Consider the follwing latent graph: 0 <- 1 -> 2 (in the joint latent space)

In [3]:
A = np.array([[0,1,0,0,0,0],
              [0,0,0,0,0,0],
              [0,-2,0,0,0,0],
              [0,0,0,0,0,0],
              [0,0,0,0,0,0],
              [0,0,0,0,0,0]])
A

array([[ 0,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0, -2,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0]])

In [10]:
I = np.identity(nlat)

#### 1) Define third order tensor of noise variables

In [11]:
Omega = np.zeros((nlat,nlat,nlat))
for i in range(nlat):
    Omega[i,i,i] = third_moms[i]

#### 2) Define mixing matrix (sampled uniformly)

In [12]:
env1_latents = joint_latents + env1_specific
env2_latents = joint_latents + env2_specific
env3_latents = joint_latents + env3_specific

In [13]:
def matrix_from_normal(shape, mu=0, sigma=1):
    A = np.random.normal(loc=mu, scale=sigma, size=shape)
    return A

In [14]:
# Sample entries of G1 from Unif([0,1])
np.random.seed(d1)
G1 = matrix_from_normal(shape=(d1,len(env1_latents)))

# Assume exaclty one pure child, i.e. one row in G1 has only one non-zero entry among the first 3 columns 
# but all other rows do not have zero entries
G1[np.ix_([0],[1,2])] = 0
G1.round(5)

array([[ 0.47299,  0.     ,  0.     , -1.70074],
       [ 0.75314, -1.53472,  0.00513, -0.12023],
       [-0.80698,  2.87182, -0.59782,  0.47246],
       [ 1.09596, -1.21517,  1.34236, -0.12215],
       [ 1.01252, -0.91387, -1.02953,  1.2098 ],
       [ 0.50187,  0.13885,  0.64076,  0.52733],
       [-1.15436, -2.21333, -1.68176, -1.78809],
       [-2.21853, -0.64743, -0.5284 , -0.03921],
       [ 0.21498, -0.38436, -0.2539 ,  0.07325],
       [-0.9972 , -0.71386,  0.03542, -0.67795],
       [-0.57188, -0.10586,  1.33583,  0.31867],
       [-0.3376 , -0.58527, -0.11492,  2.24182]])

In [15]:
B1 = np.matmul(G1, inv(I-A)[np.ix_(env1_latents,env1_latents)])
B1.round(5)

array([[ 0.47299,  0.47299,  0.     , -1.70074],
       [ 0.75314, -0.79183,  0.00513, -0.12023],
       [-0.80698,  3.26048, -0.59782,  0.47246],
       [ 1.09596, -2.80393,  1.34236, -0.12215],
       [ 1.01252,  2.15771, -1.02953,  1.2098 ],
       [ 0.50187, -0.6408 ,  0.64076,  0.52733],
       [-1.15436, -0.00418, -1.68176, -1.78809],
       [-2.21853, -1.80916, -0.5284 , -0.03921],
       [ 0.21498,  0.33843, -0.2539 ,  0.07325],
       [-0.9972 , -1.78189,  0.03542, -0.67795],
       [-0.57188, -3.34941,  1.33583,  0.31867],
       [-0.3376 , -0.69302, -0.11492,  2.24182]])

In [16]:
np.random.seed(d2)
G2 = matrix_from_normal(shape=(d2,len(env2_latents)))

# Exactly one pure child
G2[np.ix_([0],[0,2])] = 0

B2 = np.matmul(G2, inv(I-A)[np.ix_(env2_latents,env2_latents)])
B2.round(5)

array([[ 0.     ,  0.33928,  0.     , -0.50179],
       [ 0.23557,  0.66369, -1.09586, -1.08777],
       [-0.30517, -0.37773, -0.20059,  0.3552 ],
       [ 0.68952,  2.23006, -0.56498,  0.59939],
       [-0.16294,  0.07402,  0.68163,  0.01488],
       [-0.08778, -1.31328,  0.12169, -1.13744],
       [ 0.349  ,  0.82485, -1.16718,  1.4249 ],
       [ 1.49657,  6.40999, -1.81175, -1.49831],
       [-1.45014, -3.59858,  0.22726, -0.48973],
       [-0.00053,  1.09564, -0.79321,  2.0489 ],
       [ 0.60319, -2.8251 ,  1.21504, -0.66752],
       [-0.27251,  2.70949, -0.78261, -0.25032],
       [-0.30831, -2.53517,  1.97827,  0.21969],
       [ 1.89482,  0.90933, -0.19131,  1.28726],
       [-0.24688, -0.34977,  0.22272,  0.68159]])

In [17]:
np.random.seed(d3)
G3 = matrix_from_normal(shape=(d3,len(env3_latents)))

# Exactly one pure child
G3[np.ix_([0],[0,1])] = 0

B3 = np.matmul(G3, inv(I-A)[np.ix_(env3_latents,env3_latents)])
B3.round(5)

array([[ 0.     , -0.48488,  0.24244, -1.70074],
       [ 0.75314, -0.79183,  0.00513, -0.12023],
       [-0.80698,  3.26048, -0.59782,  0.47246],
       [ 1.09596, -2.80393,  1.34236, -0.12215],
       [ 1.01252,  2.15771, -1.02953,  1.2098 ],
       [ 0.50187, -0.6408 ,  0.64076,  0.52733],
       [-1.15436, -0.00418, -1.68176, -1.78809],
       [-2.21853, -1.80916, -0.5284 , -0.03921],
       [ 0.21498,  0.33843, -0.2539 ,  0.07325],
       [-0.9972 , -1.78189,  0.03542, -0.67795],
       [-0.57188, -3.34941,  1.33583,  0.31867],
       [-0.3376 , -0.69302, -0.11492,  2.24182]])

In [18]:
params = {"Omega": Omega, 
          "mixings": [B1,B2,B3], 
          "latents":[env1_latents,env2_latents,env3_latents]}

## Compute observed covariance matrix and third-order tensor

In [19]:
def get_cov_matrices(params):
    mixings = params["mixings"]
    cov_list = []
    for M in mixings:
        cov_list.append(np.matmul(M, np.transpose(M)))
    return cov_list

In [20]:
cov_list = get_cov_matrices(params)

In [21]:
def get_third_order_tensors(params):
    Omega = params["Omega"]
    mixings = params["mixings"]
    latents = params["latents"]
    tensor_list = []
    for i, M in enumerate(mixings):
        Omega_env = Omega[np.ix_(latents[i],latents[i],latents[i])]
        tucker_tensor = (Omega_env,[M,M,M])
        T = tl.tucker_to_tensor(tucker_tensor)
        tensor_list.append(T)
    return tensor_list

In [22]:
tensor_list = get_third_order_tensors(params)

In [23]:
# Check by hand: it does the same as the following code

latents_list = params["latents"]
B1 = params["mixings"][0]
d1 = B1.shape[0]

Omega1 = Omega[np.ix_(latents_list[0],latents_list[0],latents_list[0])]
T1_v2 = np.zeros((d1,d1,d1))
for i1 in range(d1):
    for i2 in range(d1):
        for i3 in range(d1):
            for j in range(len(latents_list[0])):
                T1_v2[i1,i2,i3] = T1_v2[i1,i2,i3] + Omega1[j,j,j] * B1[i1,j] * B1[i2,j] * B1[i3,j]
                
(T1_v2.round(7) == tensor_list[0].round(7)).all()

True

In [24]:
observed = {"cov": cov_list, 
            "tensor": tensor_list, 
            "nr_env": len(cov_list)}

## Identify nr of joint latent variables and joint distribution

#### 1) Identify the mixing matrices of the environments (up to signed permutation of columns)
(Use CANDECOMP/PARAFAC of TensorLy)

In [25]:
def identify_mixing(Sigma, T):
    
    # Recover the mixing matrix up to signed permutation and scaling by tensor decomposition
    rk = matrix_rank(Sigma)
    weights, factors = parafac(T, rank=rk)
    factor = factors[1] 
    
    # Remove scaling by exploiting unit variances of the noise
    Lambda = np.matmul(np.matmul(pinv(factor), Sigma), np.transpose(pinv(factor)))
    rescaler = np.diag(np.sqrt(abs(np.diag(Lambda))))
    M = np.matmul(factor, rescaler)
    return M

In [26]:
mixings = []
for i in range(observed["nr_env"]):
    mixings.append(identify_mixing(observed["cov"][i],observed["tensor"][i]))

#### 2) Identify the third moments of the noise variables (up to signed permutation)

In [27]:
def diag_tensor_to_array(T):
    d = np.zeros(T.shape[0])
    for i in range(T.shape[0]):
        d[i] = T[i,i,i]
    return d

def identify_third_moms(B, T):
    pinv_B = pinv(B)
    Omega_r = (T,[pinv_B,pinv_B,pinv_B])
    Omega_r = tl.tucker_to_tensor(Omega_r, skip_factor=None, transpose_factors=False)
    third_moms = diag_tensor_to_array(Omega_r).round(1) # This eliminates numerical errors
    return third_moms

In [28]:
third_moments = []
for i in range(observed["nr_env"]):
    third_moments.append(identify_third_moms(mixings[i],observed["tensor"][i]))
third_moments

[array([-2.,  4.,  3., -1.]),
 array([-2. , -4.5, -3. , -1. ]),
 array([ 2. , -4.2, -3. , -1. ])]

#### 3) Identify joint latent space

In [29]:
def get_joint_abs_moms(third_moments):
    S = set(abs(third_moments[0]))
    for i in range(1,len(third_moments)):
        S = S.intersection(set(abs(third_moments[i])))
    return np.array(list(S))

In [30]:
joint_abs_moms = get_joint_abs_moms(third_moments)
joint_abs_moms

array([1., 2., 3.])

In [31]:
# Reorder columns of the mixing matrix such that the columns corresponding to the joint latent space come first 
# (in same order)

def reorder_columns(B,third_moms,joint_abs_moms):
    third_abs_moms = abs(third_moms)
    nr_lat = B.shape[1]
    P = np.zeros((nr_lat,nr_lat))

    positions = []
    for i, mom in enumerate(joint_abs_moms):
        current_pos = np.where(abs(third_moms)==mom)[0][0]
        positions.append(current_pos)
        if (np.sign(third_moms[current_pos]) == -1):
            P[current_pos,i] = -1
        else:
            P[current_pos,i] = 1
    
    remaining_rows = set(np.arange(nr_lat)) - set(positions)
    
    for i, row in enumerate(remaining_rows):
        P[row,len(joint_abs_moms)+i] = 1
        
    return np.matmul(B, P)

In [32]:
for i in range(observed["nr_env"]):
    mixings[i] = reorder_columns(mixings[i], third_moments[i], joint_abs_moms)

In [33]:
mixings[0].round(5)

array([[ 0.47299, -0.47299,  0.     , -1.70074],
       [ 0.75314,  0.79183,  0.00513, -0.12023],
       [-0.80698, -3.26048, -0.59782,  0.47246],
       [ 1.09596,  2.80393,  1.34236, -0.12215],
       [ 1.01252, -2.15771, -1.02953,  1.2098 ],
       [ 0.50187,  0.6408 ,  0.64076,  0.52733],
       [-1.15436,  0.00418, -1.68176, -1.78809],
       [-2.21853,  1.80916, -0.52841, -0.03921],
       [ 0.21498, -0.33843, -0.2539 ,  0.07325],
       [-0.9972 ,  1.78189,  0.03542, -0.67795],
       [-0.57188,  3.34941,  1.33583,  0.31867],
       [-0.3376 ,  0.69302, -0.11492,  2.24182]])

In [34]:
B1.round(5)

array([[ 0.47299,  0.47299,  0.     , -1.70074],
       [ 0.75314, -0.79183,  0.00513, -0.12023],
       [-0.80698,  3.26048, -0.59782,  0.47246],
       [ 1.09596, -2.80393,  1.34236, -0.12215],
       [ 1.01252,  2.15771, -1.02953,  1.2098 ],
       [ 0.50187, -0.6408 ,  0.64076,  0.52733],
       [-1.15436, -0.00418, -1.68176, -1.78809],
       [-2.21853, -1.80916, -0.5284 , -0.03921],
       [ 0.21498,  0.33843, -0.2539 ,  0.07325],
       [-0.9972 , -1.78189,  0.03542, -0.67795],
       [-0.57188, -3.34941,  1.33583,  0.31867],
       [-0.3376 , -0.69302, -0.11492,  2.24182]])

#### 4) Define "joint mixing matrix" (up to blockwise permutation)

In [35]:
def joint_mixing_matrix(mixings, nr_joint):
    nr_domains = len(mixings)
    
    total_nr_obs = sum([M.shape[0] for M in mixings])
    total_nr_lat = sum([M.shape[1]-nr_joint for M in mixings]) + nr_joint
    M_large = np.zeros((total_nr_obs, total_nr_lat))
    
    current_row = 0
    current_col = nr_joint
    
    for M in mixings:
        nrows, ncols = M.shape
        nr_domain_specific = ncols - nr_joint
        
        M_large[np.ix_(np.arange(current_row,(current_row+nrows)),np.arange(nr_joint))] = M[:,:nr_joint]
        M_large[np.ix_(np.arange(current_row,(current_row+nrows)),
                       np.arange(current_col,(current_col+nr_domain_specific)))] = M[:,nr_joint:]
        
        current_row = current_row + nrows
        current_col = current_col + nr_domain_specific
        
    return M_large

In [36]:
# Recovered joint mixing matrix
joint_mixing_matrix(mixings,len(joint_abs_moms)).round(3)

array([[ 0.473, -0.473,  0.   , -1.701,  0.   ,  0.   ],
       [ 0.753,  0.792,  0.005, -0.12 ,  0.   ,  0.   ],
       [-0.807, -3.26 , -0.598,  0.472,  0.   ,  0.   ],
       [ 1.096,  2.804,  1.342, -0.122,  0.   ,  0.   ],
       [ 1.013, -2.158, -1.03 ,  1.21 ,  0.   ,  0.   ],
       [ 0.502,  0.641,  0.641,  0.527,  0.   ,  0.   ],
       [-1.154,  0.004, -1.682, -1.788,  0.   ,  0.   ],
       [-2.219,  1.809, -0.528, -0.039,  0.   ,  0.   ],
       [ 0.215, -0.338, -0.254,  0.073,  0.   ,  0.   ],
       [-0.997,  1.782,  0.035, -0.678,  0.   ,  0.   ],
       [-0.572,  3.349,  1.336,  0.319,  0.   ,  0.   ],
       [-0.338,  0.693, -0.115,  2.242,  0.   ,  0.   ],
       [-0.   , -0.339,  0.   ,  0.   ,  0.502,  0.   ],
       [ 0.236, -0.664, -1.095,  0.   ,  1.088,  0.   ],
       [-0.305,  0.378, -0.201,  0.   , -0.355,  0.   ],
       [ 0.689, -2.23 , -0.564,  0.   , -0.599,  0.   ],
       [-0.163, -0.074,  0.682,  0.   , -0.015,  0.   ],
       [-0.087,  1.313,  0.121,

TODO:
- Note: Recovering the true mixing matrix is getting imprecise if the third order moments of the noise are large in comaprison to the mixing matrix. (Why?) 
- Implement recovering of latent graph is there are two pure children.
- Since we have direct access to the mixing matrix: Are there milder conditions under which we can recover the graph?

## Identify joint latent graph

The following is an just some data exploration on the special case where each latent variable has exatly one pure children (and all other observed variables have a nonzero direct effect from all joint latent variables).

In [37]:
# True joint mixing matrix
B_large = joint_mixing_matrix([B1,B2,B3],len(joint_abs_moms)).round(3)
B_large

array([[ 0.473,  0.473,  0.   , -1.701,  0.   ,  0.   ],
       [ 0.753, -0.792,  0.005, -0.12 ,  0.   ,  0.   ],
       [-0.807,  3.26 , -0.598,  0.472,  0.   ,  0.   ],
       [ 1.096, -2.804,  1.342, -0.122,  0.   ,  0.   ],
       [ 1.013,  2.158, -1.03 ,  1.21 ,  0.   ,  0.   ],
       [ 0.502, -0.641,  0.641,  0.527,  0.   ,  0.   ],
       [-1.154, -0.004, -1.682, -1.788,  0.   ,  0.   ],
       [-2.219, -1.809, -0.528, -0.039,  0.   ,  0.   ],
       [ 0.215,  0.338, -0.254,  0.073,  0.   ,  0.   ],
       [-0.997, -1.782,  0.035, -0.678,  0.   ,  0.   ],
       [-0.572, -3.349,  1.336,  0.319,  0.   ,  0.   ],
       [-0.338, -0.693, -0.115,  2.242,  0.   ,  0.   ],
       [ 0.   ,  0.339,  0.   ,  0.   , -0.502,  0.   ],
       [ 0.236,  0.664, -1.096,  0.   , -1.088,  0.   ],
       [-0.305, -0.378, -0.201,  0.   ,  0.355,  0.   ],
       [ 0.69 ,  2.23 , -0.565,  0.   ,  0.599,  0.   ],
       [-0.163,  0.074,  0.682,  0.   ,  0.015,  0.   ],
       [-0.088, -1.313,  0.122,

In [38]:
# By eye: Pick sparse rows (0,12,27) and first three columns (rows and cols could be permuted)
B_large[np.ix_([0,27,12],[1,2,0])]

array([[ 0.473,  0.   ,  0.473],
       [-0.485,  0.242,  0.   ],
       [ 0.339,  0.   ,  0.   ]])

In [42]:
# Take sparset row first (one entry, has to be source node)
# Then take row with one entry in same column and one entry elsewhere (here: cannot distinguish)

B_large[np.ix_([12,0,27],[1,2,0])]

array([[ 0.339,  0.   ,  0.   ],
       [ 0.473,  0.   ,  0.473],
       [-0.485,  0.242,  0.   ]])

In [48]:
# Find the permutation of B_large such that there is a non-zero diagonal 
# (This permutation should be unique if we picked the correct permutation in the first step)

B_tilde = B_large[np.ix_([12,0,27],[1,0,2])]
B_tilde

array([[ 0.339,  0.   ,  0.   ],
       [ 0.473,  0.473,  0.   ],
       [-0.485,  0.   ,  0.242]])

#### Now, we apply the "standard" LiNGAM procedure:

In [56]:
# 1) Divide each row by its diagonal element

D = np.diag(B_tilde)
B_tilde = np.matmul(np.diag(1/D), B_tilde)
B_tilde

array([[ 1.        ,  0.        ,  0.        ],
       [ 1.        ,  1.        ,  0.        ],
       [-2.00413223,  0.        ,  1.        ]])

In [55]:
# 2) Compute an estimate B_hat of B
B_hat = np.identity(3) - inv(B_tilde) 
B_hat

array([[ 0.        ,  0.        ,  0.        ],
       [ 1.        , -0.        ,  0.        ],
       [-2.00413223,  0.        ,  0.        ]])

In [ ]:
# 3) To find a causal order, find the permutation matrix P (applied equally to both rows and columns) of B 
# which yields a matrix which is strictly lower triangular

# in our case this is B_hat